# Capacitated EOQ Model

In [255]:
# Load in the libraries that we'll use

import pandas as pd
import numpy as np
import logging

In [256]:
# The capacitated EOQ model comes from the following paper:  Anne M. Spence, Evan L. Porteus, (1987) Setup Reduction and Increased Effective Capacity. Management Science
# 33(10):1291-1301. https://doi.org/10.1287/mnsc.33.10.1291

# The model seems complicated, but is just a lot of algebra steps. There could be mistakes in this code.

# 9/4/23 Erin's suggestions to each of the to-do's:
# 1. Pandas lets us do this nicely. We can concatenate the dataframes together and then save to csv
# 2, 3, 4, sounds good!
# 5. Jupyter widgets might be the solution, or just a clearly marked code cell that says "change these values and rerun to see what happens"
# 6. Look into hosting on mybinder.org. It's free, open source, and creates a website that looks like the Jupyter interface, but 
#    runs all on binder's server (no need to install anything for students!). Only downside is that it's slow to generate the environment
#    the first time after one of us makes a new commit. 
#    Replit is likely similar. If we wanted a no-code interface, we could use Voila and lots of Jupyter widgets.
#    Handling other formats and tests are all good ideas. We should also re-organize and put in better comments.


# 9/3/23 to-do:
# 1. Create one big table of inputs and outputs. We want to add a holding cost a full cost in dollars- not percent. Print out to csv
# 2. For the rounded solution, let's save the one right above feasible and the one that is feasible
# 3. Create an ability to print out the summary stats for the solution and print
# 4. change the "total cost in the summary to Total Inv and Set Up Costs, there are other costs too.
# 5. If we can play around with things in Observable (or something else), then I would like to be able to reduce the set-up times, increase avaible regular hours and see what happens to the solutions
# 6. Long-term:  We need to host, try out Replit?, put in other formats, develop unit tests, and put in a bunch of catches for bad data.

In [257]:
logging.basicConfig(level=logging.INFO)

## Model Inputs

Feel free to modify these to understand how the model works. Then `Run All`. 

In [258]:
#This cell is for reading system wide inputs from a csv file.
# We will be reading specific cells in a spreadsheet

system_input_table = pd.read_csv('system_variables_v1.csv', header=None)
total_hours = system_input_table.iloc[0, 1]
avail_regular_hours = system_input_table.iloc[1, 1]

avail_time_constraint = avail_regular_hours / total_hours  #expressed as a percentage

inv_holding_cost_percent = system_input_table.iloc[2, 1]
cost_of_set_up_hour = system_input_table.iloc[3, 1]

system_input_table #display the table
print("Total Hours: ", total_hours)
print("Available Regular Hours: ", avail_regular_hours)
print("Available Time Constraint: ", avail_time_constraint)
print("Inventory Holding Cost Percent: ", inv_holding_cost_percent)
print("Cost of Set Up Hour: ", cost_of_set_up_hour)

Total Hours:  8736.0
Available Regular Hours:  4264.0
Available Time Constraint:  0.4880952380952381
Inventory Holding Cost Percent:  0.2
Cost of Set Up Hour:  100.0


In [259]:
#This cell is for reading the product-specific inputs

product_input_table = pd.read_csv("product_input_v2.csv", index_col="Product ID")

nominal_set_up_time = product_input_table['Set Up Time'].min()  # finds the minimum set up time.  This will be the nominal set up time.  
#I think the paper uses nominal time to make it easy to change for what-ifs

print(product_input_table) # display the table
print("Nominal Set Up Time: ", nominal_set_up_time) 

            Demand (year)- m  Cost- c-j  Production Rate (hr)  Set Up Time
Product ID                                                                
1                       5000        100                    15           10
2                       5000        200                    15           10
3                       8000        200                    15           20
4                       5000        400                    15           20
5                       5000        400                    15           30
6                       5000        500                    15           30
7                       5000        200                    15           40
8                       5000        500                    15           40
9                       5000        500                    15           50
10                      5000       1000                    15           50
Nominal Set Up Time:  10


In [260]:
product_input_table["Set Up Scaler-- (q-j)"] = product_input_table["Set Up Time"] / nominal_set_up_time
product_input_table["unit_set_up_cost"] = product_input_table["Set Up Time"] * cost_of_set_up_hour
product_input_table

,Demand (year)- m,Cost- c-j,Production Rate (hr),Set Up Time,Set Up Scaler-- (q-j),unit_set_up_cost
Product ID,,,,,,
1,5000,100,15,10,1.0,1000.0
2,5000,200,15,10,1.0,1000.0
3,8000,200,15,20,2.0,2000.0
4,5000,400,15,20,2.0,2000.0
5,5000,400,15,30,3.0,3000.0
6,5000,500,15,30,3.0,3000.0
7,5000,200,15,40,4.0,4000.0
8,5000,500,15,40,4.0,4000.0
9,5000,500,15,50,5.0,5000.0


## Internal calculations

In [261]:
# Rename the input table so that formulas are shorter and more readable

inputs_df = product_input_table.copy()
inputs_df.columns = ["demand", "cost", "rate", "set_up_time", "scalar", "set_up_cost"]

inputs_df

,demand,cost,rate,set_up_time,scalar,set_up_cost
Product ID,,,,,,
1,5000,100,15,10,1.0,1000.0
2,5000,200,15,10,1.0,1000.0
3,8000,200,15,20,2.0,2000.0
4,5000,400,15,20,2.0,2000.0
5,5000,400,15,30,3.0,3000.0
6,5000,500,15,30,3.0,3000.0
7,5000,200,15,40,4.0,4000.0
8,5000,500,15,40,4.0,4000.0
9,5000,500,15,50,5.0,5000.0


In [262]:
#create nasty calculation table
#create system-wide paramters  they will be mixed and matched depending on what needs to happen

cogs = (inputs_df["demand"] * inputs_df["cost"]).sum()

sys_S_nomimal_set_up_years = nominal_set_up_time / total_hours
sys_c_S_cost_of_set_up_years = cost_of_set_up_hour / total_hours * total_hours * total_hours #this was in my spreadsheet like this. need to figure out why


internal_calc_df = pd.DataFrame()  
internal_calc_df["Prod_rate_yr_r_j"] = inputs_df["rate"] * total_hours
internal_calc_df["m_over_r"] = inputs_df["demand"] / internal_calc_df["Prod_rate_yr_r_j"]
internal_calc_df["sqrt_2mqic_j"] = np.sqrt(2 * inputs_df["demand"] * inputs_df["scalar"] * inv_holding_cost_percent *inputs_df["cost"] )

c_sum = internal_calc_df["sqrt_2mqic_j"].sum()
alpha = internal_calc_df["m_over_r"].sum()
if alpha > avail_time_constraint:  #this means the problem is infeasible-- production time takes longer than available time
    infeasible_flag = True 
else:
    infeasible_flag = False
    
lambda_var = (c_sum **2 * sys_S_nomimal_set_up_years) / (4*(avail_time_constraint - alpha)**2) - sys_c_S_cost_of_set_up_years 

x = sys_c_S_cost_of_set_up_years
y = sys_S_nomimal_set_up_years
internal_calc_df["Q-1 (S)"] = np.sqrt((2*inputs_df["demand"] * inputs_df["scalar"] * x * y)/(inv_holding_cost_percent * inputs_df["cost"]))
internal_calc_df["Q-2"] = ((c_sum * y) / (avail_time_constraint - alpha)) * np.sqrt((inputs_df["demand"] * inputs_df["scalar"]) / (2 * inv_holding_cost_percent * inputs_df["cost"])) 

print("sys_S_nomimal_set_up_years",sys_S_nomimal_set_up_years)
print("sys_c_S_cost_of_set_up_years",sys_c_S_cost_of_set_up_years)
print("alpha",alpha)
print("cogs",cogs)
print("lambda_var",lambda_var)
internal_calc_df.astype("float").round(3)


sys_S_nomimal_set_up_years 0.0011446886446886447
sys_c_S_cost_of_set_up_years 873600.0
alpha 0.40445665445665446
cogs 20600000
lambda_var 8854056.919427026


,Prod_rate_yr_r_j,m_over_r,sqrt_2mqic_j,Q-1 (S),Q-2
Product ID,,,,,
1,131040.0,0.038,447.214,707.107,2359.570
2,131040.0,0.038,632.456,500.000,1668.468
3,131040.0,0.061,1131.371,894.427,2984.646
4,131040.0,0.038,1264.911,500.000,1668.468
5,131040.0,0.038,1549.193,612.372,2043.447
6,131040.0,0.038,1732.051,547.723,1827.715
7,131040.0,0.038,1264.911,1000.000,3336.935
8,131040.0,0.038,2000.000,632.456,2110.463
9,131040.0,0.038,2236.068,707.107,2359.570


In [263]:
c_sum

15420.451889286827

## Functions

In [264]:
# All the functions we plan on using

## Model

In [265]:
# Optimal Results of capaciated model

optimal_df = pd.DataFrame() 

if lambda_var <= 0:
    optimal_df["q_star_order_size"] = internal_calc_df["Q-1 (S)"]
else:
    optimal_df["q_star_order_size"] = internal_calc_df["Q-2"]

optimal_df["opt_setups_per_year"] = inputs_df["demand"] / optimal_df["q_star_order_size"]
optimal_df["opt_time_in_setups_per_year"] = inputs_df["demand"] * inputs_df["scalar"] * sys_S_nomimal_set_up_years / optimal_df["q_star_order_size"] 

opt_total_setup_percent = optimal_df["opt_time_in_setups_per_year"].sum()

#The effective set up cost is what the set up cost would need to be if the order size was calculated using an uncapacited EOQ model. 
#In other words, it takes into account the "opportunity cost" of the limited capacity. I'm not sure it is the full opportunity cost because we might need some revenue numbers too
optimal_df["effective_setup_cost"] = (np.square(optimal_df["q_star_order_size"])) * (inv_holding_cost_percent * inputs_df["cost"]) / (2 * inputs_df["demand"])

print("opt_total_setup_percent",opt_total_setup_percent)
optimal_df.astype("float").round(4)

opt_total_setup_percent 0.08363858363858362


,q_star_order_size,opt_setups_per_year,opt_time_in_setups_per_year,effective_setup_cost
Product ID,,,,
1,2359.5697,2.1190,0.0024,11135.1384
2,1668.4677,2.9968,0.0034,11135.1384
3,2984.6458,2.6804,0.0061,22270.2768
4,1668.4677,2.9968,0.0069,22270.2768
5,2043.4473,2.4468,0.0084,33405.4152
6,1827.7148,2.7357,0.0094,33405.4152
7,3336.9355,1.4984,0.0069,44540.5537
8,2110.4633,2.3691,0.0108,44540.5537
9,2359.5697,2.1190,0.0121,55675.6921


In [266]:
# EOQ, Unconstrained Results -- theorhetical best, but not feasible

eoq_df = pd.DataFrame() 

# Redefine x and y for readability
x = sys_c_S_cost_of_set_up_years
y = sys_S_nomimal_set_up_years
eoq_df["q_star_eoq"] = np.sqrt((2 * inputs_df["demand"] * inputs_df["scalar"] * y * x) / (inv_holding_cost_percent * inputs_df["cost"] ))

eoq_df["eoq_setups_per_year"] = inputs_df["demand"] / eoq_df["q_star_eoq"]
eoq_df["eoq_time_in_setups_per_year"] = inputs_df["demand"] * inputs_df["scalar"] * sys_S_nomimal_set_up_years / eoq_df["q_star_eoq"] 

eoq_total_setup_percent = eoq_df["eoq_time_in_setups_per_year"].sum()

print("eoq_total_setup_percent",eoq_total_setup_percent)
eoq_df.astype("float").round(4)

eoq_total_setup_percent 0.2790965574587512


,q_star_eoq,eoq_setups_per_year,eoq_time_in_setups_per_year
Product ID,,,
1,707.1068,7.0711,0.0081
2,500.0000,10.0000,0.0114
3,894.4272,8.9443,0.0205
4,500.0000,10.0000,0.0229
5,612.3724,8.1650,0.0280
6,547.7226,9.1287,0.0313
7,1000.0000,5.0000,0.0229
8,632.4555,7.9057,0.0362
9,707.1068,7.0711,0.0405


In [267]:
# This is filling out the cost table, both EOQ and Optimal

cost_df = pd.DataFrame() 

#for optimal
cost_df["opt setup cost"] = (inputs_df["demand"] * inputs_df["scalar"] * y * x) / optimal_df["q_star_order_size"]
cost_df["opt hold cost"] = optimal_df["q_star_order_size"] * inv_holding_cost_percent * inputs_df["cost"] / 2

#for EOQ
cost_df["eoq setup cost"] = (inputs_df["demand"] * inputs_df["scalar"] * y * x) / eoq_df["q_star_eoq"]
cost_df["eoq hold cost"] = eoq_df["q_star_eoq"] * inv_holding_cost_percent * inputs_df["cost"] / 2

cost_df.astype("float").round(0)

,opt setup cost,opt hold cost,eoq setup cost,eoq hold cost
Product ID,,,,
1,2119.0,23596.0,7071.0,7071.0
2,2997.0,33369.0,10000.0,10000.0
3,5361.0,59693.0,17889.0,17889.0
4,5994.0,66739.0,20000.0,20000.0
5,7341.0,81738.0,24495.0,24495.0
6,8207.0,91386.0,27386.0,27386.0
7,5994.0,66739.0,20000.0,20000.0
8,9477.0,105523.0,31623.0,31623.0
9,10595.0,117978.0,35355.0,35355.0


## Results

In [268]:
# And here are the results!

print("----- Summary Stats ----------")
print("")
print("------Optimal Plan -----------")

total_holding_inventory_cost = round(cost_df['opt hold cost'].sum())
total_setup_cost = round(cost_df['opt setup cost'].sum())
total_cost = round(total_holding_inventory_cost + total_setup_cost)
avg_working_cap = round(total_holding_inventory_cost / inv_holding_cost_percent)
inv_turns = cogs / avg_working_cap
hours_used = total_hours * (optimal_df["opt_time_in_setups_per_year"].sum() + alpha)
if hours_used <= avail_regular_hours:
    overtime = 0
else:
    overtime = hours_used - avail_regular_hours


print(f"Total Holding Inventory Cost: ${total_holding_inventory_cost:>10,.0f}")
print(f"Total Setup Cost:             ${total_setup_cost:>10,.0f}")
print(f"Total Cost:                   ${total_cost:>10,.0f}")
print("")
print(f"Average Working Capital       ${avg_working_cap:>10,.0f}")
print(f"Inventory Turns               {inv_turns:>10,.1f}")
print("")
print(f"Total Hours- all time          {total_hours:>10,.0f}")
print(f"Total Working Hours            {avail_regular_hours:>10,.0f}")
print(f"Working Hours Used             {hours_used:>10,.0f}")
print(f"Overtime Needed                {overtime:>10,.0f}")

print("")
print("------EOQ Plan (Theoretical Best) -----------")

total_holding_inventory_cost = round(cost_df['eoq hold cost'].sum())
total_setup_cost = round(cost_df['eoq setup cost'].sum())
total_cost = round(total_holding_inventory_cost + total_setup_cost)
avg_working_cap = round(total_holding_inventory_cost / inv_holding_cost_percent)
inv_turns = cogs / avg_working_cap
hours_used = total_hours * (eoq_df["eoq_time_in_setups_per_year"].sum() + alpha)
if hours_used <= avail_regular_hours:
    overtime = 0
else:
    overtime = hours_used - avail_regular_hours


print(f"Total Holding Inventory Cost: ${total_holding_inventory_cost:>10,.0f}")
print(f"Total Setup Cost:             ${total_setup_cost:>10,.0f}")
print(f"Total Cost:                   ${total_cost:>10,.0f}")
print("")
print(f"Average Working Capital       ${avg_working_cap:>10,.0f}")
print(f"Inventory Turns               {inv_turns:>10,.1f}")
print("")
print(f"Total Hours- all time          {total_hours:>10,.0f}")
print(f"Total Working Hours            {avail_regular_hours:>10,.0f}")
print(f"Working Hours Used             {hours_used:>10,.0f}")
print(f"Overtime Needed                {overtime:>10,.0f}")




----- Summary Stats ----------

------Optimal Plan -----------
Total Holding Inventory Cost: $   813,607
Total Setup Cost:             $    73,067
Total Cost:                   $   886,674

Average Working Capital       $ 4,068,035
Inventory Turns                      5.1

Total Hours- all time               8,736
Total Working Hours                 4,264
Working Hours Used                  4,264
Overtime Needed                         0

------EOQ Plan (Theoretical Best) -----------
Total Holding Inventory Cost: $   243,819
Total Setup Cost:             $   243,819
Total Cost:                   $   487,638

Average Working Capital       $ 1,219,095
Inventory Turns                     16.9

Total Hours- all time               8,736
Total Working Hours                 4,264
Working Hours Used                  5,972
Overtime Needed                     1,708


## Round to nearest power of 2

Take the optimal solution, round the number of set ups per year to the nearest power of two, then recalculate the solution.

In [269]:
# Remember what the table looks like
optimal_df.round(4)

,q_star_order_size,opt_setups_per_year,opt_time_in_setups_per_year,effective_setup_cost
Product ID,,,,
1,2359.5697,2.1190,0.0024,11135.1384
2,1668.4677,2.9968,0.0034,11135.1384
3,2984.6458,2.6804,0.0061,22270.2768
4,1668.4677,2.9968,0.0069,22270.2768
5,2043.4473,2.4468,0.0084,33405.4152
6,1827.7148,2.7357,0.0094,33405.4152
7,3336.9355,1.4984,0.0069,44540.5537
8,2110.4633,2.3691,0.0108,44540.5537
9,2359.5697,2.1190,0.0121,55675.6921


In [270]:
# round to nearest power of 2

def round_to_power_of_2(x):
    """Round x to the nearest power of 2. Returns the power of 2.
    For example, 3.2 rounds to 4.
    Break ties going up."""

    if np.isnan(x):
        # Non-number input
        return np.nan
    elif np.log2(x) % 1 == 0:
        # Already a power of 2
        return np.log2(x)
    else:
        # Round
        lower = 2 ** np.floor(np.log2(x))
        upper = 2 ** np.ceil(np.log2(x))

        if upper - x <= x - lower:
            return upper
        else:
            return lower
        


In [271]:
# Test the function

round_to_power_of_2(50)

64.0

In [272]:
# Build the rounded table
rounded_df = pd.DataFrame()

# Round the number of setups to the nearest power of 2 per product
rounded_df["rounded_setups_per_year"] = optimal_df["opt_setups_per_year"].apply(round_to_power_of_2)

# Calculate the new order size
rounded_df["order_size"] = inputs_df["demand"] / rounded_df["rounded_setups_per_year"]

# Calculate the new time in setups per year
rounded_df["time_in_setups_per_year"] = inputs_df["demand"] * inputs_df["scalar"] * sys_S_nomimal_set_up_years / rounded_df["order_size"]


rounded_df # not a very interesting example dataset!

,rounded_setups_per_year,order_size,time_in_setups_per_year
Product ID,,,
1,2.0,2500.0,0.002289
2,2.0,2500.0,0.002289
3,2.0,4000.0,0.004579
4,2.0,2500.0,0.004579
5,2.0,2500.0,0.006868
6,2.0,2500.0,0.006868
7,1.0,5000.0,0.004579
8,2.0,2500.0,0.009158
9,2.0,2500.0,0.011447


In [273]:
# Add to the cost table

# Redeclare x and y for readability
x = sys_c_S_cost_of_set_up_years
y = sys_S_nomimal_set_up_years

cost_df["rounded setup cost"] = (inputs_df["demand"] * inputs_df["scalar"] * y * x) / rounded_df["order_size"]
cost_df["rounded hold cost"] = rounded_df["order_size"] * inv_holding_cost_percent * inputs_df["cost"] / 2

cost_df.astype("float").round(0)

,opt setup cost,opt hold cost,eoq setup cost,eoq hold cost,rounded setup cost,rounded hold cost
Product ID,,,,,,
1,2119.0,23596.0,7071.0,7071.0,2000.0,25000.0
2,2997.0,33369.0,10000.0,10000.0,2000.0,50000.0
3,5361.0,59693.0,17889.0,17889.0,4000.0,80000.0
4,5994.0,66739.0,20000.0,20000.0,4000.0,100000.0
5,7341.0,81738.0,24495.0,24495.0,6000.0,100000.0
6,8207.0,91386.0,27386.0,27386.0,6000.0,125000.0
7,5994.0,66739.0,20000.0,20000.0,4000.0,100000.0
8,9477.0,105523.0,31623.0,31623.0,8000.0,125000.0
9,10595.0,117978.0,35355.0,35355.0,10000.0,125000.0


In [274]:
# Print out the new results!

print("-------------- Summary Stats --------------")
print("")
print("---- Rounded version of Optimal plan ----\n")

# aggregate directly from table
total_holding_inventory_cost = round(cost_df['rounded hold cost'].sum())
total_setup_cost = round(cost_df['rounded setup cost'].sum())
total_cost = round(total_holding_inventory_cost + total_setup_cost)

# calculate from table
avg_working_cap = round(total_holding_inventory_cost / inv_holding_cost_percent)
inv_turns = cogs / avg_working_cap
hours_used = total_hours * (rounded_df["time_in_setups_per_year"].sum() + alpha)
if hours_used <= avail_regular_hours:
    overtime = 0
    logging.debug("No overtime needed")
else:
    overtime = hours_used - avail_regular_hours
    logging.debug("Overtime needed")


print(f"Total Holding Inventory Cost: ${total_holding_inventory_cost:>10,.0f}")
print(f"Total Setup Cost:             ${total_setup_cost:>10,.0f}")
print(f"Total Cost:                   ${total_cost:>10,.0f}")
print("")
print(f"Average Working Capital       ${avg_working_cap:>10,.0f}")
print(f"Inventory Turns               {inv_turns:>10,.1f}")
print("")
print(f"Total Hours- all time          {total_hours:>10,.0f}")
print(f"Total Working Hours            {avail_regular_hours:>10,.0f}")
print(f"Working Hours Used             {hours_used:>10,.0f}")
print(f"Overtime Needed                {overtime:>10,.0f}")

# Print the optimal plan again

print("\n------------ Optimal Plan ------------\n")

# aggregate directly from table
total_holding_inventory_cost = round(cost_df['opt hold cost'].sum())
total_setup_cost = round(cost_df['opt setup cost'].sum())
total_cost = round(total_holding_inventory_cost + total_setup_cost)

# calculate from table
avg_working_cap = round(total_holding_inventory_cost / inv_holding_cost_percent)
inv_turns = cogs / avg_working_cap
hours_used = total_hours * (optimal_df["opt_time_in_setups_per_year"].sum() + alpha)
if hours_used <= avail_regular_hours:
    overtime = 0
    logging.debug("No overtime needed")
else:
    overtime = hours_used - avail_regular_hours
    logging.debug("Overtime needed")


print(f"Total Holding Inventory Cost: ${total_holding_inventory_cost:>10,.0f}")
print(f"Total Setup Cost:             ${total_setup_cost:>10,.0f}")
print(f"Total Cost:                   ${total_cost:>10,.0f}")
print("")
print(f"Average Working Capital       ${avg_working_cap:>10,.0f}")
print(f"Inventory Turns               {inv_turns:>10,.1f}")
print("")
print(f"Total Hours- all time          {total_hours:>10,.0f}")
print(f"Total Working Hours            {avail_regular_hours:>10,.0f}")
print(f"Working Hours Used             {hours_used:>10,.0f}")
print(f"Overtime Needed                {overtime:>10,.0f}")

# Print the EOQ plan again

print("\n------------ EOQ Plan ------------\n")

# aggregate directly from table
total_holding_inventory_cost = round(cost_df['eoq hold cost'].sum())
total_setup_cost = round(cost_df['eoq setup cost'].sum())
total_cost = round(total_holding_inventory_cost + total_setup_cost)

# calculate from table
avg_working_cap = round(total_holding_inventory_cost / inv_holding_cost_percent)
inv_turns = cogs / avg_working_cap
hours_used = total_hours * (eoq_df["eoq_time_in_setups_per_year"].sum() + alpha)
if hours_used <= avail_regular_hours:
    overtime = 0
    logging.debug("No overtime needed")
else:
    overtime = hours_used - avail_regular_hours
    logging.debug("Overtime needed")


print(f"Total Holding Inventory Cost: ${total_holding_inventory_cost:>10,.0f}")
print(f"Total Setup Cost:             ${total_setup_cost:>10,.0f}")
print(f"Total Cost:                   ${total_cost:>10,.0f}")
print("")
print(f"Average Working Capital       ${avg_working_cap:>10,.0f}")
print(f"Inventory Turns               {inv_turns:>10,.1f}")
print("")
print(f"Total Hours- all time          {total_hours:>10,.0f}")
print(f"Total Working Hours            {avail_regular_hours:>10,.0f}")
print(f"Working Hours Used             {hours_used:>10,.0f}")
print(f"Overtime Needed                {overtime:>10,.0f}")




-------------- Summary Stats --------------

---- Rounded version of Optimal plan ----

Total Holding Inventory Cost: $ 1,080,000
Total Setup Cost:             $    56,000
Total Cost:                   $ 1,136,000

Average Working Capital       $ 5,400,000
Inventory Turns                      3.8

Total Hours- all time               8,736
Total Working Hours                 4,264
Working Hours Used                  4,093
Overtime Needed                         0

------------ Optimal Plan ------------

Total Holding Inventory Cost: $   813,607
Total Setup Cost:             $    73,067
Total Cost:                   $   886,674

Average Working Capital       $ 4,068,035
Inventory Turns                      5.1

Total Hours- all time               8,736
Total Working Hours                 4,264
Working Hours Used                  4,264
Overtime Needed                         0

------------ EOQ Plan ------------

Total Holding Inventory Cost: $   243,819
Total Setup Cost:             $   

## Same number for all

In [275]:
def hours_for_all_same_setup(num_setups):
    """Calculate the number of hours needed for a given number of setups,
    when we force all products to use the same number of setups."""

    logging.debug(f"Calculating hours for {num_setups} setups")

    # Calculate the order size
    order_sizes = inputs_df["demand"] / num_setups

    # Calculate the time in setups per year
    time_in_setups_per_year = inputs_df["demand"] * inputs_df["scalar"] * sys_S_nomimal_set_up_years / order_sizes
    logging.debug(f"time_in_setups_per_year: {time_in_setups_per_year}")

    # Calculate the total hours for this number of setups
    total_hours_used = total_hours * (time_in_setups_per_year.sum() + alpha)

    return total_hours_used
     

In [276]:
# Find the largest number of setups used for all products 
# that still fits within the available time

# Start with the largest number of setups in the EOQ
num_setups = round(eoq_df["eoq_setups_per_year"].max())

hours_required = hours_for_all_same_setup(num_setups)

while hours_required > avail_regular_hours:
    num_setups -= 1
    hours_required = hours_for_all_same_setup(num_setups)

    print(f"Number of Setups: {num_setups}")
    print(f"Hours Required:  {hours_required:>10,.0f}")
    print(f"Hours Available: {avail_regular_hours:>10,.0f}")    
    temp_hours_over = hours_required - avail_regular_hours
    print(f"Hours Over:      {temp_hours_over:>10,.0f}")
    print()

Number of Setups: 9
Hours Required:       6,233
Hours Available:      4,264
Hours Over:           1,969

Number of Setups: 8
Hours Required:       5,933
Hours Available:      4,264
Hours Over:           1,669

Number of Setups: 7
Hours Required:       5,633
Hours Available:      4,264
Hours Over:           1,369

Number of Setups: 6
Hours Required:       5,333
Hours Available:      4,264
Hours Over:           1,069

Number of Setups: 5
Hours Required:       5,033
Hours Available:      4,264
Hours Over:             769

Number of Setups: 4
Hours Required:       4,733
Hours Available:      4,264
Hours Over:             469

Number of Setups: 3
Hours Required:       4,433
Hours Available:      4,264
Hours Over:             169

Number of Setups: 2
Hours Required:       4,133
Hours Available:      4,264
Hours Over:            -131

